In [16]:
# -*- coding: utf-8 -*
import os
import re
import shutil
import textract
import numpy as np
import pandas as pd
# NLTK
from nltk import tokenize
from nltk import SnowballStemmer
from nltk.corpus import stopwords
# Import punctuation marks
from string import punctuation
from tqdm.notebook import tqdm
from PyPDF2 import PdfFileReader

SEED = 42

## Leitura file_description

In [80]:
def identifica_atas_df(file_description, root_dir):
    # Create flags for word ata
    file_description['ata_from_url'] = 0
    file_description['ata_from_file_name'] = 0
    file_description['ata_from_file_title'] = 0
    file_description['ata_from_file_content'] = 0
    # Check urls that contains the word ata
    ata_regex = 'ata|Ata|aTa|atA|ATA'
    file_description.loc[file_description.url.str.split("/").str[-1].str.contains(ata_regex, regex=True), 'ata_from_file_name'] = 1
    file_description.loc[file_description.url.str.contains(ata_regex, regex=True), 'ata_from_url'] = 1
    # Check file content
    for idx, row in tqdm(file_description.iterrows()):
        try:
            pdf = PdfFileReader(os.path.join(root_dir, row['file_name']))
        except Exception as e:
            print("{0}: {1}".format(row['file_name'],e))
        if pdf is None:
            print(row['file_name'])
            pass
        try:
            pdf_num_pages = pdf.getNumPages()
            pdf_doc_info = pdf.getDocumentInfo()
        except:
            pass
        # Procurar ocorrencias da string "ata" nos títulos
        try:
            # Procurar a string nos títulos como lowercase, para evitar problemas de matching
            if 'ata' in pdf_doc_info['/Title'].lower():
                file_description.loc[idx, 'ata_from_file_title'] = 1
        except:
            pass
        # read only the first 5 pages
        pages_to_read = pdf_num_pages if pdf_num_pages <= 5 else 5
        for i in range(pages_to_read):
            try:
                page = pdf.getPage(i).extractText().lower().replace('\n', '')
            except Exception:
                pass
            if re.search(r'\bata\b', page):
                file_description.loc[idx, 'ata_from_file_content'] = 1
                break
    return file_description

In [105]:
def extrai_possiveis_atas(files_dir, file_description, num_atas, num_nao_atas):
    # Usando o .sum() aqui pois como os valores da coluna são 0 ou 1,
    # achei que ficava mais claro para ver em quantos docs a gente
    # pode identificar uma ata. O .size() tava meio confuso pra mim.
    print(file_description[[
        'ata_from_url',
        'ata_from_file_name',
        'ata_from_file_title',
        'ata_from_file_content'
    ]].sum())

    # Seleciona aleatoriamente amostras de arquivos
    filter_ = file_description[[
        'ata_from_url',
        'ata_from_file_name',
        'ata_from_file_title',
        'ata_from_file_content'
    ]].sum(axis=1)


    # Não foi detectada nenhuma presença da palavra ata
    nao_contem_ata = file_description.loc[(filter_ == 0) & (file_description['verified'] == 0), 'file_name'].sample(num_nao_atas, random_state=SEED).tolist()
    # Detectada alguma presença da palavra ata
    contem_ata = file_description.loc[filter_ > 0 & (file_description['verified'] == 0), 'file_name'].sample(num_atas, random_state=SEED).tolist()
    # Marca arquivo ja visitado
    file_description.loc[file_description['file_name'].isin(contem_ata+nao_contem_ata), 'verified'] = 1
    # Onde foi encontrada a palavra ata
    file_description.loc[file_description['file_name'].isin(contem_ata), ['file_name',
                                       'ata_from_url',
                                       'ata_from_file_name',
                                       'ata_from_file_title',
                                       'ata_from_file_content']]

    # Copia os arquivos sorteados para a pasta download
    # a pasta será compactada e baixada para análise local
    if os.path.exists(os.path.join(files_dir, "download")):
        shutil.rmtree(os.path.join(files_dir, "download"))
    os.makedirs(os.path.join(files_dir, "download"), exist_ok=True)
    all_files = contem_ata + nao_contem_ata
    for file in all_files:
        src = os.path.join(files_dir, file)
        dst = os.path.join(files_dir, "download", file)    
        shutil.copy(src, dst)

    # Remove arquivo caso exista
    try:
        ! rm {os.path.join(files_dir, "download.zip")}
    except:
        pass
    # Compacta arquivo
    ! cd {files_dir} && zip -r "download.zip" "download" && cd -
    
    return file_description

# Coqueiral

In [67]:
files_dir_coqueiral = "../data/289-licitacoes-coqueiral/data/files/"
file_description_coqueiral = pd.read_json(os.path.join(files_dir_coqueiral,"file_description.jsonl"), lines=True)
file_description_coqueiral = file_description_coqueiral.loc[file_description_coqueiral['type'] == 'pdf']
file_description_coqueiral = identifica_atas_df(file_description_coqueiral)
file_description_coqueiral['verified'] = 0
file_description_coqueiral.head()

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
0,https://www.coqueiral.mg.gov.br/public/storage...,2ba855845281501e57212fba9606c873.pdf,1,16076483502732,2020-12-11 01:00:31.840464,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,0,0
1,https://www.coqueiral.mg.gov.br/public/storage...,0b96de38bae73ce0cc9f749d4732ee1a.pdf,1,16076483502732,2020-12-11 01:01:07.498196,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,0
2,https://www.coqueiral.mg.gov.br/public/storage...,c452358bc51d7b3995f3b974668edf8b.pdf,1,16076483502732,2020-12-11 01:01:11.998517,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,0,0
3,https://www.coqueiral.mg.gov.br/public/storage...,2b2ea1d5c0576cd25fb9dfdccf67d7bb.pdf,1,16076483502732,2020-12-11 01:01:19.631454,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,0
4,https://www.coqueiral.mg.gov.br/public/storage...,fdc2fa2d183e90d75cf6b5cab04f076e.pdf,1,16076483502732,2020-12-11 01:01:24.298670,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,0,0


In [72]:
file_description_coqueiral = extrai_possiveis_atas(files_dir_coqueiral, file_description_coqueiral, num_atas=60, num_nao_atas=0)
file_description_coqueiral.loc[file_description_coqueiral['verified'] == 1]

ata_from_url               3
ata_from_file_name         3
ata_from_file_title        0
ata_from_file_content    669
dtype: int64
updating: download/ (stored 0%)
updating: download/91fd36339080b510682d37550da7ee19.pdf (deflated 3%)
updating: download/cbd9fcfa1bf2680dc0d7caf9b421bb6c.pdf (deflated 6%)
updating: download/045b9d0f9616a19a45fb233b07e0e224.pdf (deflated 8%)
updating: download/111de6912446591310a42aea3efdccde.pdf (deflated 6%)
updating: download/40cd778b345466e020967a2d4ed5c170.pdf (deflated 6%)
updating: download/7a0bf8e951833cfcb5106536341b4575.pdf (deflated 10%)
updating: download/a1b7069244b09363f907d5c2b00d00c0.pdf (deflated 7%)
updating: download/a880bb3b88c6e72b49a007aef2c846f8.pdf (deflated 9%)
updating: download/f271c4973bc50315f1179925a8b3bdc4.pdf (deflated 5%)
updating: download/b64e298a9ab9ef4b502db5b82d2c183c.pdf (deflated 8%)
updating: download/65c40f3ee50408f41a785af3fe552640.pdf (deflated 11%)
updating: download/eec00d9659b9f0b9cc0c680703cbcbe2.pdf (deflated 8

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
20,https://www.coqueiral.mg.gov.br/public/storage...,f3cfe81ac31dfe738742a6ccc765f7b9.pdf,1,16076483502732,2020-12-11 01:03:25.106314,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[],0,0,0,1,1
57,https://www.coqueiral.mg.gov.br/public/storage...,a1089ad9ac9fb0407bf0f5f6124ab5d2.pdf,1,16076483502732,2020-12-11 01:07:45.016372,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
134,https://www.coqueiral.mg.gov.br/public/storage...,c9f4c9ee1c4c40dbf056e3d8d9fa15bd.pdf,1,16076483502732,2020-12-11 01:17:49.906802,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
135,https://www.coqueiral.mg.gov.br/public/storage...,6aa821e1e916cac34caf106284ff38b8.pdf,1,16076483502732,2020-12-11 01:18:04.446850,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
159,https://www.coqueiral.mg.gov.br/public/storage...,f90b4c2c5f66a4b81408489e58a45a14.pdf,1,16076483502732,2020-12-11 01:22:04.382913,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[],0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,https://www.coqueiral.mg.gov.br/public/editor/...,b64e298a9ab9ef4b502db5b82d2c183c.pdf,1,16076483502732,2020-12-11 04:11:55.019735,https://www.coqueiral.mg.gov.br/licitacoes/doc...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,0,1
1491,https://www.coqueiral.mg.gov.br/public/storage...,e5b96e2f42b1d05693aae50b9ff72598.pdf,1,16076483502732,2020-12-11 04:13:49.307383,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
1515,https://www.coqueiral.mg.gov.br/public/storage...,22001864b1538899920db84f8179f06c.pdf,1,16076483502732,2020-12-11 04:16:47.356698,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
1517,https://www.coqueiral.mg.gov.br/public/storage...,edeb3816c2e3ae18951f769ecda9cfb8.pdf,1,16076483502732,2020-12-11 04:16:58.796807,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[],0,0,0,0,1


# São Bento Abade

In [67]:
files_dir_sao_bento_abade = "../data/381-licitacoes-sao-bento-abade/data/files/"
file_description_sao_bento_abade = pd.read_json(os.path.join(files_dir_sao_bento_abade,"file_description.jsonl"), lines=True)
file_description_sao_bento_abade = file_description_sao_bento_abade.loc[file_description_sao_bento_abade['type'] == 'pdf']
file_description_sao_bento_abade = identifica_atas_df(file_description_sao_bento_abade)
file_description_sao_bento_abade['verified'] = 0
file_description_sao_bento_abade.head()

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
0,https://www.coqueiral.mg.gov.br/public/storage...,2ba855845281501e57212fba9606c873.pdf,1,16076483502732,2020-12-11 01:00:31.840464,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,0,0
1,https://www.coqueiral.mg.gov.br/public/storage...,0b96de38bae73ce0cc9f749d4732ee1a.pdf,1,16076483502732,2020-12-11 01:01:07.498196,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,0
2,https://www.coqueiral.mg.gov.br/public/storage...,c452358bc51d7b3995f3b974668edf8b.pdf,1,16076483502732,2020-12-11 01:01:11.998517,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,0,0
3,https://www.coqueiral.mg.gov.br/public/storage...,2b2ea1d5c0576cd25fb9dfdccf67d7bb.pdf,1,16076483502732,2020-12-11 01:01:19.631454,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,0
4,https://www.coqueiral.mg.gov.br/public/storage...,fdc2fa2d183e90d75cf6b5cab04f076e.pdf,1,16076483502732,2020-12-11 01:01:24.298670,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,0,0


In [72]:
file_description_sao_bento_abade = extrai_possiveis_atas(files_dir_sao_bento_abade, file_description_sao_bento_abade, num_atas=60, num_nao_atas=0)
file_description_sao_bento_abade.loc[file_description_sao_bento_abade['verified'] == 1]

ata_from_url               3
ata_from_file_name         3
ata_from_file_title        0
ata_from_file_content    669
dtype: int64
updating: download/ (stored 0%)
updating: download/91fd36339080b510682d37550da7ee19.pdf (deflated 3%)
updating: download/cbd9fcfa1bf2680dc0d7caf9b421bb6c.pdf (deflated 6%)
updating: download/045b9d0f9616a19a45fb233b07e0e224.pdf (deflated 8%)
updating: download/111de6912446591310a42aea3efdccde.pdf (deflated 6%)
updating: download/40cd778b345466e020967a2d4ed5c170.pdf (deflated 6%)
updating: download/7a0bf8e951833cfcb5106536341b4575.pdf (deflated 10%)
updating: download/a1b7069244b09363f907d5c2b00d00c0.pdf (deflated 7%)
updating: download/a880bb3b88c6e72b49a007aef2c846f8.pdf (deflated 9%)
updating: download/f271c4973bc50315f1179925a8b3bdc4.pdf (deflated 5%)
updating: download/b64e298a9ab9ef4b502db5b82d2c183c.pdf (deflated 8%)
updating: download/65c40f3ee50408f41a785af3fe552640.pdf (deflated 11%)
updating: download/eec00d9659b9f0b9cc0c680703cbcbe2.pdf (deflated 8

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
20,https://www.coqueiral.mg.gov.br/public/storage...,f3cfe81ac31dfe738742a6ccc765f7b9.pdf,1,16076483502732,2020-12-11 01:03:25.106314,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[],0,0,0,1,1
57,https://www.coqueiral.mg.gov.br/public/storage...,a1089ad9ac9fb0407bf0f5f6124ab5d2.pdf,1,16076483502732,2020-12-11 01:07:45.016372,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
134,https://www.coqueiral.mg.gov.br/public/storage...,c9f4c9ee1c4c40dbf056e3d8d9fa15bd.pdf,1,16076483502732,2020-12-11 01:17:49.906802,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
135,https://www.coqueiral.mg.gov.br/public/storage...,6aa821e1e916cac34caf106284ff38b8.pdf,1,16076483502732,2020-12-11 01:18:04.446850,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
159,https://www.coqueiral.mg.gov.br/public/storage...,f90b4c2c5f66a4b81408489e58a45a14.pdf,1,16076483502732,2020-12-11 01:22:04.382913,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[],0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,https://www.coqueiral.mg.gov.br/public/editor/...,b64e298a9ab9ef4b502db5b82d2c183c.pdf,1,16076483502732,2020-12-11 04:11:55.019735,https://www.coqueiral.mg.gov.br/licitacoes/doc...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,0,1
1491,https://www.coqueiral.mg.gov.br/public/storage...,e5b96e2f42b1d05693aae50b9ff72598.pdf,1,16076483502732,2020-12-11 04:13:49.307383,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
1515,https://www.coqueiral.mg.gov.br/public/storage...,22001864b1538899920db84f8179f06c.pdf,1,16076483502732,2020-12-11 04:16:47.356698,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...,0,0,0,1,1
1517,https://www.coqueiral.mg.gov.br/public/storage...,edeb3816c2e3ae18951f769ecda9cfb8.pdf,1,16076483502732,2020-12-11 04:16:58.796807,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[],0,0,0,0,1


# Cristais

In [78]:
file_description_cristais

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
0,https://www.cristais.mg.gov.br/portal/download...,f1a9cef9fa8609613a16d230dbdb7c6f.pdf,2,160762462001740,2020-12-10 18:24:58.997044,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0
1,https://www.cristais.mg.gov.br/portal/download...,6d1cfa73e9217808bf4fc6384294f460.pdf,2,160762462001740,2020-12-10 18:25:02.892016,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0
2,https://www.cristais.mg.gov.br/portal/download...,20ae297764fe60f322fd3ed8e9f27666.pdf,2,160762462001740,2020-12-10 18:25:01.125206,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0
3,https://www.cristais.mg.gov.br/portal/download...,5272e20a7771cead26d153bc040fcc69.pdf,2,160762462001740,2020-12-10 18:25:37.143052,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0
4,https://www.cristais.mg.gov.br/portal/download...,4b58aa2968a032d7be82b0049dc5245a.pdf,2,160762462001740,2020-12-10 18:25:38.310340,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1833,https://www.cristais.mg.gov.br/portal/download...,0c5eabd2871ef0c8d52bc28c93a1c211.pdf,2,160762462001740,2020-12-10 20:09:30.192820,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0
1834,https://www.cristais.mg.gov.br/portal/download...,3b7dc6582368510f82bb9e69662f3dfc.pdf,2,160762462001740,2020-12-10 20:09:35.026447,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0
1835,https://www.cristais.mg.gov.br/portal/download...,7834ff090167647a5aee3a737726e141.pdf,2,160762462001740,2020-12-10 20:09:32.828525,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0
1836,https://www.cristais.mg.gov.br/portal/download...,b2ae6cddd299c0e124c28e3aa788a0a1.pdf,2,160762462001740,2020-12-10 20:09:33.253130,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0


In [81]:
files_dir_cristais = "../data/290-licitacoes-cristais/data/files/"
file_description_cristais = pd.read_json(os.path.join(files_dir_cristais,"file_description.jsonl"), lines=True)

file_description_cristais['type'] = file_description_cristais['type'].str.replace(';','')
file_description_cristais['file_name'] = file_description_cristais['file_name'].str.replace(';','')

file_description_cristais = file_description_cristais.loc[file_description_cristais['type'] == 'pdf']
file_description_cristais = identifica_atas_df(file_description_cristais, files_dir_cristais)
file_description_cristais['verified'] = 0
file_description_cristais.head()

678ebb1cc0b5b9ca358a2b14450c840a.pdf: Cannot read an empty file
827a398f255b73d083d073b4e3aed5cf.pdf: Cannot read an empty file
8ce63c93769d62d1c716f27b4e7dbf10.pdf: Could not find xref table at specified location
45cc0a2aff268eb41ccb07427c46d90f.pdf: Could not find xref table at specified location
dfc237ffd10c1b6bc62fe5aac5137977.pdf: Could not find xref table at specified location
d4e4037f2275159b54f08fa7d66ec028.pdf: Could not find xref table at specified location


,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
0,https://www.cristais.mg.gov.br/portal/download...,f1a9cef9fa8609613a16d230dbdb7c6f.pdf,2,160762462001740,2020-12-10 18:24:58.997044,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,0
1,https://www.cristais.mg.gov.br/portal/download...,6d1cfa73e9217808bf4fc6384294f460.pdf,2,160762462001740,2020-12-10 18:25:02.892016,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,0
2,https://www.cristais.mg.gov.br/portal/download...,20ae297764fe60f322fd3ed8e9f27666.pdf,2,160762462001740,2020-12-10 18:25:01.125206,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,0
3,https://www.cristais.mg.gov.br/portal/download...,5272e20a7771cead26d153bc040fcc69.pdf,2,160762462001740,2020-12-10 18:25:37.143052,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,0
4,https://www.cristais.mg.gov.br/portal/download...,4b58aa2968a032d7be82b0049dc5245a.pdf,2,160762462001740,2020-12-10 18:25:38.310340,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,1,0


In [83]:
file_description_cristais = extrai_possiveis_atas(files_dir_cristais, file_description_cristais, num_atas=0, num_nao_atas=20)
file_description_cristais.loc[file_description_cristais['verified'] == 1]

ata_from_url               0
ata_from_file_name         0
ata_from_file_title      133
ata_from_file_content    745
dtype: int64
  adding: download/ (stored 0%)
  adding: download/a60c9fbd82d570e34c638657e002a3cc.pdf (deflated 30%)
  adding: download/ccd8bafbda7d9bd0770f26cd5856ac83.pdf (deflated 12%)
  adding: download/60dce8fb5e668056d831377ad8a529ee.pdf (deflated 16%)
  adding: download/726d5e4738eec3654b7be9cdb15eb856.pdf (deflated 9%)
  adding: download/fbd732e5ee481ff62391a151bf9ec3a5.pdf (deflated 24%)
  adding: download/3ae3254b55d6a6dba62b52b507caa758.pdf (deflated 16%)
  adding: download/087624708ee92b58a4983d4541f7342c.pdf (deflated 17%)
  adding: download/5746e74d1ea7c0107e99bb89c9b84266.pdf (deflated 2%)
  adding: download/0c5eabd2871ef0c8d52bc28c93a1c211.pdf (deflated 19%)
  adding: download/b82e44f20dd581d8f553ba96e7b0cc8f.pdf (deflated 16%)
  adding: download/8888d6bda6dfa34e4591ed71e2407704.pdf (deflated 19%)
  adding: download/b2ae6cddd299c0e124c28e3aa788a0a1.pdf (def

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
51,https://www.cristais.mg.gov.br/portal/download...,6222628428f24c3accd10c82eed1867e.pdf,2,160762462001740,2020-12-10 18:27:42.177626,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,1
66,https://www.cristais.mg.gov.br/portal/download...,1ceb0ed6af83c2ced20539c2b4dcfb50.pdf,2,160762462001740,2020-12-10 18:28:19.253603,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,1,1
86,https://www.cristais.mg.gov.br/portal/download...,bfbecee6caa49264379579968753e5e3.pdf,2,160762462001740,2020-12-10 18:29:54.085907,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,1,1
104,https://www.cristais.mg.gov.br/portal/download...,31f78f9fc0a229a460b25074d9ffe858.pdf,2,160762462001740,2020-12-10 18:30:39.139560,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,1,1
110,https://www.cristais.mg.gov.br/portal/download...,48e5407965b7c21e5ec1a8ecd2359e9d.pdf,2,160762462001740,2020-12-10 18:30:48.593363,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743,https://www.cristais.mg.gov.br/portal/download...,3ae3254b55d6a6dba62b52b507caa758.pdf,2,160762462001740,2020-12-10 20:04:38.168100,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,1
1778,https://www.cristais.mg.gov.br/portal/download...,d5d7c273d8adc585aae620cd6809fcff.pdf,2,160762462001740,2020-12-10 20:06:50.358329,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,1,1
1786,https://www.cristais.mg.gov.br/portal/download...,a60c9fbd82d570e34c638657e002a3cc.pdf,2,160762462001740,2020-12-10 20:07:09.886077,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,1
1833,https://www.cristais.mg.gov.br/portal/download...,0c5eabd2871ef0c8d52bc28c93a1c211.pdf,2,160762462001740,2020-12-10 20:09:30.192820,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN,0,0,0,0,1


# Passa Vinte

In [84]:
files_dir_passa_vinte = "../data/304-licitacoes-passa-vinte/data/files/"
file_description_passa_vinte = pd.read_json(os.path.join(files_dir_passa_vinte,"file_description.jsonl"), lines=True)

file_description_passa_vinte['type'] = file_description_passa_vinte['type'].str.replace(';','')
file_description_passa_vinte['file_name'] = file_description_passa_vinte['file_name'].str.replace(';','')

file_description_passa_vinte = file_description_passa_vinte.loc[file_description_passa_vinte['type'] == 'pdf']
file_description_passa_vinte = identifica_atas_df(file_description_passa_vinte, files_dir_passa_vinte)
file_description_passa_vinte['verified'] = 0
file_description_passa_vinte.head()

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,time_between_downloads,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
0,https://www.passavinte.mg.gov.br/download/down...,4547a6d7529ac939fcbc5b579bd64c40.pdf,5,160746828499708,2020-12-08 23:06:39.720399,https://www.passavinte.mg.gov.br/processo-no-1...,pdf,NaN,[licitacoes_passa_vinte/data/csv/4547a6d7529ac...,0,0,0,0,0
1,https://www.passavinte.mg.gov.br/download/down...,0b1985fc3b17b7401eaeac6d6b95ca2c.pdf,5,160746828499708,2020-12-08 23:07:58.806685,https://www.passavinte.mg.gov.br/processo-20-2...,pdf,NaN,[licitacoes_passa_vinte/data/csv/0b1985fc3b17b...,0,0,0,0,0
2,https://www.passavinte.mg.gov.br/download/down...,83fa2991f65f1e7d7df105b85b12fc04.pdf,5,160746828499708,2020-12-08 23:08:23.468321,https://www.passavinte.mg.gov.br/processo-lici...,pdf,NaN,[licitacoes_passa_vinte/data/csv/83fa2991f65f1...,0,0,0,0,0
3,https://www.passavinte.mg.gov.br/download/down...,cd019d89436cbe6884ce0c6a166ea9e1.pdf,5,160746828499708,2020-12-08 23:13:15.976820,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/cd019d89436cb...,0,0,0,0,0
4,https://www.passavinte.mg.gov.br/download/down...,074d45ccafdf94b6e509d8259dfcd295.pdf,5,160746828499708,2020-12-08 23:14:18.830225,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/074d45ccafdf9...,0,0,0,0,0


In [91]:
file_description_passa_vinte = extrai_possiveis_atas(files_dir_passa_vinte, file_description_passa_vinte, num_atas=40, num_nao_atas=20)
file_description_passa_vinte.loc[file_description_passa_vinte['verified'] == 1]

ata_from_url               2
ata_from_file_name         0
ata_from_file_title        0
ata_from_file_content    144
dtype: int64
  adding: download/ (stored 0%)
  adding: download/b0ccb6e4b152136415533e001ca2695c.pdf (deflated 8%)
  adding: download/e4e6fe425796b4331b1504aca21317bc.pdf (deflated 3%)
  adding: download/c1c7a083e91c964ccf6fd10fd27269e9.pdf (deflated 10%)
  adding: download/0cda7425e6bd682e0c862cc63a95abc9.pdf (deflated 4%)
  adding: download/81db7fc014f8658f8b1b06969bf2c3ae.pdf (deflated 8%)
  adding: download/cb350eb7127f65cce02e283e6017f6ee.pdf (deflated 3%)
  adding: download/e4a629d33f7956bb363a9638357b275d.pdf (deflated 16%)
  adding: download/dbd56c08e0405cd48c76edce9191bd21.pdf (deflated 3%)
  adding: download/57aaf80bffa98129fae6b2931fa7e69b.pdf (deflated 17%)
  adding: download/8a2c3cb858982aa52442e1953945080c.pdf (deflated 10%)
  adding: download/e374628598d6b7114d89552737d8e9ff.pdf (deflated 17%)
  adding: download/7e5085e0fb47f23e0428421d97c3e62b.pdf (deflate

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,time_between_downloads,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
8,https://www.passavinte.mg.gov.br/download/down...,52c37e73c962935d7bf8460d05b2ed4f.pdf,5,160746828499708,2020-12-08 23:18:42.792584,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/52c37e73c9629...,0,0,0,0,1
13,https://www.passavinte.mg.gov.br/download/cliq...,266fb4fb89150a66f2461231043e4b00.pdf,5,160746828499708,2020-12-08 23:21:21.184942,https://www.passavinte.mg.gov.br/download/cliq...,pdf,NaN,[licitacoes_passa_vinte/data/csv/266fb4fb89150...,0,0,0,0,1
20,https://www.passavinte.mg.gov.br/download/cliq...,3f3edf7f5b2c3fa3102d066e396109db.pdf,5,160746828499708,2020-12-08 23:25:47.756074,https://www.passavinte.mg.gov.br/download/cliq...,pdf,NaN,[],0,0,0,1,1
24,https://www.passavinte.mg.gov.br/download/down...,407ddef334bb539e99c27e29e222b215.pdf,5,160746828499708,2020-12-08 23:29:16.404438,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/407ddef334bb5...,0,0,0,0,1
36,https://www.passavinte.mg.gov.br/download/down...,d3193a43ddd0ea8d5b186efe3c2f148e.pdf,5,160751772286184,2020-12-09 12:51:45.959408,https://www.passavinte.mg.gov.br/processo-45-2...,pdf,NaN,[licitacoes_passa_vinte/data/csv/d3193a43ddd0e...,0,0,0,1,1
37,https://www.passavinte.mg.gov.br/download/down...,6a6ed441afcb8d9826f17dfdda06ca9f.pdf,5,160751772286184,2020-12-09 13:00:47.961699,https://www.passavinte.mg.gov.br/processo-106/,pdf,NaN,[licitacoes_passa_vinte/data/csv/6a6ed441afcb8...,0,0,0,0,1
38,https://www.passavinte.mg.gov.br/download/down...,ac6bdf515e63ec7560b71627dcdc2c4a.pdf,5,160751772286184,2020-12-09 13:01:12.051659,https://www.passavinte.mg.gov.br/processo-no-2...,pdf,NaN,[licitacoes_passa_vinte/data/csv/ac6bdf515e63e...,0,0,0,1,1
40,https://www.passavinte.mg.gov.br/download/down...,fef4b8f3a92171b5884a97d72c2fbd89.pdf,5,160751772286184,2020-12-09 13:02:51.610255,https://www.passavinte.mg.gov.br/processo-52-2...,pdf,NaN,[licitacoes_passa_vinte/data/csv/fef4b8f3a9217...,0,0,0,1,1
42,https://www.passavinte.mg.gov.br/download/down...,e374628598d6b7114d89552737d8e9ff.pdf,5,160751772286184,2020-12-09 13:03:54.266356,https://www.passavinte.mg.gov.br/processo-no-1...,pdf,NaN,[licitacoes_passa_vinte/data/csv/e374628598d6b...,0,0,0,1,1
54,https://www.passavinte.mg.gov.br/download/cliq...,0045b7363686bfa0bd0cedb4a6ec7d25.pdf,5,160751772286184,2020-12-09 13:12:35.244625,https://www.passavinte.mg.gov.br/download/cliq...,pdf,NaN,[],0,0,0,1,1


# Pirapetinga

In [95]:
files_dir_pirapetinga = "../data/288_licitacoes-pirapetinga/data/files/"
file_description_pirapetinga = pd.read_json(os.path.join(files_dir_pirapetinga,"file_description.jsonl"), lines=True)

file_description_pirapetinga['type'] = file_description_pirapetinga['type'].str.replace(';','')
file_description_pirapetinga['file_name'] = file_description_pirapetinga['file_name'].str.replace(';','')

file_description_pirapetinga = file_description_pirapetinga.loc[file_description_pirapetinga['type'] == 'application/pdf']
file_description_pirapetinga = identifica_atas_df(file_description_pirapetinga, files_dir_pirapetinga)
file_description_pirapetinga['verified'] = 0
file_description_pirapetinga.head()

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
0,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 21:58:50.994079,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,bd9cf430f53cfa701f8c7d51e2e0ba2f.pdf,application/pdf,[],0,0,0,0,0
1,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 21:58:50.994526,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,bc0866e315473780ab8d83b14278998f.pdf,application/pdf,[],0,0,0,0,0
2,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.249297,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,c7bd8928fa0ff4856199e1eb2589e8dc.pdf,application/pdf,[],0,0,0,0,0
3,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.249461,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,25bff9c51ff33ab73b3ec2a5c1ae8ea7.pdf,application/pdf,[],0,0,0,0,0
4,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.248959,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,5df4600b8c2d6bfc9fbdb9034a937966.pdf,application/pdf,[],0,0,0,0,0


In [99]:
file_description_pirapetinga = extrai_possiveis_atas(files_dir_pirapetinga, file_description_pirapetinga.loc[file_description_pirapetinga.verified ==0], num_atas=40, num_nao_atas=20)
file_description_pirapetinga.loc[file_description_pirapetinga['verified'] == 1]

ata_from_url               0
ata_from_file_name         0
ata_from_file_title        0
ata_from_file_content    129
dtype: int64


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:966]


updating: download/ (stored 0%)
  adding: download/aa080311449fccb7fd6b5c57503aa8fb.pdf (deflated 8%)
  adding: download/13a05bf3ddfbf6667529ecfc32d2234d.pdf (deflated 11%)
  adding: download/610fb51955bc23897eb6b349dfb74a4e.pdf (deflated 4%)
  adding: download/48de864931f8104786dd9fc1efc6ad92.pdf (deflated 9%)
  adding: download/f9b498137795c6fb3f3e13cb88cec26d.pdf (deflated 8%)
  adding: download/3adefd7e77c57c8b129744fd20a3c06d.pdf (deflated 4%)
  adding: download/44b0a1a230db3e3de889ad5568f7124e.pdf (deflated 8%)
  adding: download/099cad41ef0f2c30dae708eb0e530763.pdf (deflated 5%)
  adding: download/f12e7df15d66ef2c3932b875154334ac.pdf (deflated 7%)
  adding: download/584857a9de6bc391d932fe1a6b73e97f.pdf (deflated 7%)
  adding: download/24a1bb6c216209a40530b36734d8b527.pdf (deflated 4%)
  adding: download/86b62a304ebbfcababe727417578a4fc.pdf (deflated 4%)
  adding: download/f2ae6b32414474b95cd5d30f256a68f4.pdf (deflated 5%)
  adding: download/f482c817b5caf06e2a7e9844a9e405b7.pdf (

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type,extracted_files,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
24,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:03:06.413708,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,45cbedef69a379637cafa687a913a702.pdf,application/pdf,[/home/elves/coletores/pirapetinga/data/csv/45...,0,0,0,1,1
32,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:03:23.036635,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,f482c817b5caf06e2a7e9844a9e405b7.pdf,application/pdf,[/home/elves/coletores/pirapetinga/data/csv/f4...,0,0,0,1,1
86,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:13:07.001965,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,8b7c93c099b74ca1562bc0a5cda03bb9.pdf,application/pdf,[],0,0,0,0,1
97,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:13:26.295505,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,8d5f91f9850ee41d15f4cbaf1275459d.pdf,application/pdf,[/home/elves/coletores/pirapetinga/data/csv/8d...,0,0,0,1,1
99,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:13:26.294483,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,0bc6fd39d568bfebcd281064ff9b1051.pdf,application/pdf,[/home/elves/coletores/pirapetinga/data/csv/0b...,0,0,0,1,1
109,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:13:57.837682,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,d98eab10e285839ecfe3e1450c399ca0.pdf,application/pdf,[/home/elves/coletores/pirapetinga/data/csv/d9...,0,0,0,1,1
128,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:15:10.576008,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,c8ea49b5a4b874579f52affa746f6560.pdf,application/pdf,[/home/elves/coletores/pirapetinga/data/csv/c8...,0,0,0,1,1
152,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:18:07.975275,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,d20631a7a29a74e2796d7f364163ee3d.pdf,application/pdf,[],0,0,0,0,1
207,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:24:04.136392,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,f2655d1214fa60d93bf345a3d2697eae.pdf,application/pdf,[],0,0,0,0,1
307,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:30:55.173245,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,1b7f39600117b2301361e74f10b1b58d.pdf,application/pdf,[],0,0,0,0,1


# Arantina

In [106]:
files_dir_arantina = "../data/306-licitacoes-arantina/data/files/"
file_description_arantina = pd.read_json(os.path.join(files_dir_arantina,"file_description.jsonl"), lines=True)

file_description_arantina['type'] = file_description_arantina['type'].str.replace(';','')
file_description_arantina['file_name'] = file_description_arantina['file_name'].str.replace(';','')

file_description_arantina = file_description_arantina.loc[file_description_arantina['type'] == 'pdf']
file_description_arantina = identifica_atas_df(file_description_arantina, files_dir_arantina)
file_description_arantina['verified'] = 0
file_description_arantina.head()

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
0,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:30.175044,https://www.arantina.mg.gov.br/licitacao2020,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf,0,0,0,1,0
1,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:34.236158,https://www.arantina.mg.gov.br/leis2020,False,1,be439ad0f779d00dbf96310c07c58abb.pdf,pdf,0,0,0,0,0
2,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:34.237156,https://www.arantina.mg.gov.br/leis2020,False,1,6630277a946046b0e70cbdbf314d51f4.pdf,pdf,0,0,0,0,0
3,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:32.706535,https://www.arantina.mg.gov.br/copia-dispensa-...,False,1,c387044bcb423b95c007a43208102e8d.pdf,pdf,0,0,0,0,0
4,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:34.240611,https://www.arantina.mg.gov.br/leis2020,False,1,aeea94794d0f556d5c97ff93bf76e18c.pdf,pdf,0,0,0,0,0


In [107]:
file_description_arantina = extrai_possiveis_atas(files_dir_arantina, file_description_arantina.loc[file_description_arantina.verified ==0], num_atas=40, num_nao_atas=20)
file_description_arantina.loc[file_description_arantina['verified'] == 1]

ata_from_url               0
ata_from_file_name         0
ata_from_file_title        0
ata_from_file_content    204
dtype: int64
  adding: download/ (stored 0%)
  adding: download/11911196c980f90a1abc6bdad3a63973.pdf (deflated 9%)
  adding: download/c0021dd9bce3969ec38409c3f7a838e4.pdf (deflated 9%)
  adding: download/cab4db9211862cc058cb550f76fa902e.pdf (deflated 8%)
  adding: download/1d1a6ad9e052b2b2d52d8da38424d828.pdf (deflated 9%)
  adding: download/a471cc3b8c720f7ee029301cdb6fb9e6.pdf (deflated 6%)
  adding: download/c7a57eab550b0f554220e5e9019e261e.pdf (deflated 10%)
  adding: download/b848f8ab40ed59dba275be9883b0bd0c.pdf (deflated 9%)
  adding: download/19344b012b9900770b7d219406a7bfe0.pdf (deflated 9%)
  adding: download/a394f5dadda36a930be8b13fad91b33f.pdf (deflated 8%)
  adding: download/c62cf83f96b2d2ef529c251d8a01b183.pdf (deflated 9%)
  adding: download/a6783931751d6d144ae922d31c4a3d6b.pdf (deflated 8%)
  adding: download/a9088141081197f11ba3d91f244aa2ca.pdf (deflated 7%

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content,verified
0,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:30.175044,https://www.arantina.mg.gov.br/licitacao2020,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf,0,0,0,1,1
39,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:34.236773,https://www.arantina.mg.gov.br/leis2020,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf,0,0,0,1,1
60,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:36.440537,https://www.arantina.mg.gov.br/tomadadepreco2020,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf,0,0,0,1,1
62,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:32.710968,https://www.arantina.mg.gov.br/copia-dispensa-...,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf,0,0,0,1,1
72,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:41.202029,https://www.arantina.mg.gov.br/copia-licitacao...,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:32:05.427160,https://www.arantina.mg.gov.br/chamada-publica...,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf,0,0,0,1,1
959,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:31:51.185340,https://www.arantina.mg.gov.br/copia-convite-2016,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf,0,0,0,1,1
964,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:31:09.749205,https://www.arantina.mg.gov.br/pregao2019,False,1,c93d6e6502c82aec0e94e626f346e1d8.pdf,pdf,0,0,0,0,1
972,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:32:19.635370,https://www.arantina.mg.gov.br/copia-pregao-2017,False,1,28e5f2bade386279fe9e22391df8d368.pdf,pdf,0,0,0,0,1
